In [24]:
import pandas as pd

In [25]:
import warnings
warnings.filterwarnings(action='ignore')

In [26]:
from pyspark.sql import SparkSession

In [27]:
from pyspark.context import SparkContext

In [28]:
spark = SparkSession.builder.appName("tess").getOrCreate()

In [29]:
from pyspark.sql.functions import col, pandas_udf

In [30]:
from pyspark.sql.types import LongType

In [31]:
def cubed(a: pd.Series) -> pd.Series:
    return a*a*a

In [32]:
cubed_udf = pandas_udf(cubed, returnType=LongType())

In [33]:
x = pd.Series([1,2,3])
print(cubed(x))

0     1
1     8
2    27
dtype: int64


In [34]:
df = spark.range(1,4)

In [35]:
df.select("id",cubed_udf(col("id"))).show()

+---+---------+
| id|cubed(id)|
+---+---------+
|  1|        1|
|  2|        8|
|  3|       27|
+---+---------+



In [36]:
from platform import python_version
print(python_version())


3.7.13


In [37]:
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# define schema for our data
schema = StructType([
   StructField("Id", IntegerType(), False),
   StructField("First", StringType(), False),
   StructField("Last", StringType(), False),
   StructField("Url", StringType(), False),
   StructField("Published", StringType(), False),
   StructField("Hits", IntegerType(), False),
   StructField("Campaigns", ArrayType(StringType()), False)])

#create our data
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
       [2, "Brooke","Wenig","https://tinyurl.2", "5/5/2018", 8908, ["twitter", "LinkedIn"]],
       [3, "Denny", "Lee", "https://tinyurl.3","6/7/2019",7659, ["web", "twitter", "FB", "LinkedIn"]],
       [4, "Tathagata", "Das","https://tinyurl.4", "5/12/2018", 10568, ["twitter", "FB"]],
       [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web", "twitter", "FB", "LinkedIn"]],
       [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568, ["twitter", "LinkedIn"]]
      ]
# main program
if __name__ == "__main__":
   #create a SparkSession
   spark = (SparkSession
       .builder
       .appName("Example-3_6")
       .getOrCreate())
   # create a DataFrame using the schema defined above
   blogs_df = spark.createDataFrame(data, schema)
   # show the DataFrame; it should reflect our table above
   blogs_df.show()
   print()
   # print the schema used by Spark to process the DataFrame
   print(blogs_df.printSchema())
   # Show columns and expressions
   blogs_df.select(expr("Hits") * 2).show(2)
   blogs_df.select(col("Hits") * 2).show(2)
   blogs_df.select(expr("Hits * 2")).show(2)
   # show heavy hitters
   blogs_df.withColumn("Big Hitters", (expr("Hits > 10000"))).show()
   print(blogs_df.schema)

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+


root
 |-- Id: integer (nullable = false)
 |-- First: string (nullable = false)
 |-- Last: string (nullable = false)
 |-- Url: string (nullable = false)
 |-- Published: string (nullable = false)
 |-- Hits: int

In [38]:
spark.sql("SET -v").select("key","value").show()

+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|spark.sql.adaptiv...|<value of spark.s...|
|spark.sql.adaptiv...|                true|
|spark.sql.adaptiv...|         <undefined>|
|spark.sql.adaptiv...|         <undefined>|
|spark.sql.adaptiv...|               false|
|spark.sql.adaptiv...|                true|
|spark.sql.adaptiv...|         <undefined>|
|spark.sql.adaptiv...|                true|
|spark.sql.adaptiv...|                   5|
|spark.sql.adaptiv...|               256MB|
|spark.sql.ansi.en...|               false|
|spark.sql.autoBro...|                10MB|
|spark.sql.avro.co...|              snappy|
|spark.sql.avro.de...|                  -1|
|spark.sql.avro.fi...|                true|
|spark.sql.broadca...|                 300|
|spark.sql.bucketi...|               false|
|spark.sql.bucketi...|                   4|
|spark.sql.cache.s...|org.apache.spark....|
|spark.sql.catalog...|         <

In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [40]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

22/11/28 14:13:05 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [42]:
from pyspark.sql import SparkSession
from common_function import get_mongo_info
import urllib

ModuleNotFoundError: No module named 'common_function'

In [ ]:
spark = SparkSession \
    .builder \
    .appName("kafka_streaming") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")  # 이거 없으면 INFO메시지가 너무 많이 뜬다. 궁금하면 주석처리하고 실행해보자.

kafka_username, kafka_password = '', ''
kafka_topic_name = ''
kafka_bootstrap_server = ''
options = {
    "kafka.sasl.jaas.config": 'org.apache.kafka.common.security.plain.PlainLoginModule required username="{}" password="{}";'.format(kafka_username, kafka_password),
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol": "SASL_PLAINTEXT",
    "kafka.bootstrap.servers": kafka_bootstrap_server,
    "group.id": 'test',
    "subscribe": kafka_topic_name,
}
df = spark.readStream.format("kafka").options(**options).load()

def write_mongo_row(df, epoch_id):
    if df.rdd.isEmpty():
        return
    # df.show()  # 테스트용

    mongo_host, mongo_port, mongo_id, mongo_password, mongo_uri_format = get_mongo_info()
    mongo_uri_format = mongo_uri_format.format(mongo_id, urllib.parse.quote(mongo_password), mongo_host, mongo_port,
                                               'db.streaming_test')
    # mongo_uri_format = "mongodb://{몽고 id}:{몽고 pw}@{몽고 address}:{몽고 port}/{몽고 db name}.{몽고 collection name}"  # uri포맷은 이와 같다.
    df.write \
        .format("com.mongodb.spark.sql.DefaultSource") \
        .option("spark.mongodb.output.uri", '{}?authSource=admin'.format(mongo_uri_format)) \
        .mode("append") \
        .save()

query = df \
    .selectExpr("CAST(value AS STRING)") \
    .writeStream \
    .foreachBatch(write_mongo_row) \
    .start()

# query = df \
#     .selectExpr("*") \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()  # 콘솔로 출력하는 예제. kafka같은 스트리밍으로 들어오는 데이터의 outputMode를 complete로 하면 에러뜨면서 안됨.

query.awaitTermination()